In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os
import scipy
import numpy as np
import glob
from scipy.stats import mannwhitneyu
from matplotlib.pyplot import subplot_mosaic as mosaic
import matplotlib.transforms as mtransforms
import math
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

## Preparations

In [2]:
cwd = os.getcwd()
alpha = 0.05
cohorts = ['brca_metabric']#, 'COAD', 'GBM', 'HNSC', 'KIRC', 'KIRP', 'LUSC', 'PCPG', 'READ', 'STAD']
variables = ['stage', 'cigarettes_per_day.exposures', 'alcohol_history.exposures']

## Prepare data frames for plot showing results of the first part of the test protocol

In [5]:
def align(pheno, expr):
    keep = pheno.index.isin(expr.index)
    pheno = pheno[keep]
    pheno = pheno[pheno.index.isin(expr.index)]
    samples = pheno.index            
    expr = expr.loc[samples]
    expr = expr.loc[:, (expr.std() != 0)]
    pheno = pheno[pheno.index.isin(expr.index)]
    return pheno, expr

def get_conf_partition(pheno_data_orig, confounder_selector, rank, thresh=0, verbose=False):
    pheno_data = pheno_data_orig.copy()
    indices = None
    blocks = []
    printer = []
    pheno_field = ''  
    if confounder_selector == 'age':
        pheno_field = 'age_at_initial_pathologic_diagnosis'
        if 'age_at_initial_pathologic_diagnosis' not in list(pheno_data.columns):
            pheno_field = 'age'
    elif confounder_selector == 'ethnicity':
        pheno_field = 'race.demographic'
        if 'race.demographic' not in list(pheno_data.columns):
            pheno_field = 'ethnicity'
    elif confounder_selector == 'sex':
        pheno_field = 'gender.demographic'
        if 'gender.demographic' not in list(pheno_data.columns):
            pheno_field = 'sex'
    elif confounder_selector == 'stage':
        pheno_field = 'tumor_stage.diagnoses'
        if 'tumor_stage.diagnoses' not in list(pheno_data.columns):
            pheno_field = 'stage'
        pheno_data = pheno_data[pheno_data[pheno_field] != 'stage x']
        pheno_data.loc[pheno_data['tumor_stage.diagnoses'].astype('string').str.strip().isin(['stage ia', 'stage ib', 'stage ic']), pheno_field] = 'stage i'
        pheno_data.loc[pheno_data['tumor_stage.diagnoses'].astype('string').str.strip().isin(['stage iia', 'stage iib', 'stage iic']), pheno_field] = 'stage ii'
        pheno_data.loc[pheno_data['tumor_stage.diagnoses'].astype('string').str.strip().isin(['stage iiia', 'stage iiib', 'stage iiic', 'stage iv', 'stage iva', 'stage ivb', 'stage ivc']), pheno_field] = 'stage iii'
    elif confounder_selector == 'type':
        pheno_field = 'cohort'
    else:
        pheno_field = confounder_selector
    print('na:')
    print(len(pheno_data[pheno_data[pheno_field].isna()])+len(pheno_data[pheno_data[pheno_field] == 'not reported']))
    pheno_data = pheno_data[pheno_data[pheno_field].notna()]
    pheno_data = pheno_data[pheno_data[pheno_field] != 'not reported']
    if confounder_selector != 'age' and confounder_selector != 'cigarettes_per_day.exposures':
        blocks = sorted(list(set(pheno_data[pheno_field].astype('string').str.strip().values)))
        for block_attr in blocks:
            samples = pheno_data.loc[pheno_data[pheno_field].astype('string').str.strip() == block_attr].index.tolist()
            if len(samples) >= thresh:
                printer.append((samples, block_attr))
    elif confounder_selector == 'age' or confounder_selector == 'cigarettes_per_day.exposures':       
        samples_lower = []
        samples_upper = []
        for cohort in set(pheno_data['cohort'].str.strip().values):
            pheno_cohort = pheno_data[pheno_data['cohort'] == cohort]
            pheno_cohort = pheno_cohort[pheno_cohort[pheno_field] != 'na']
            lower, upper = pheno_cohort[pheno_field].quantile(0.25), pheno_cohort[pheno_field].quantile(0.75)
            if verbose:
                print(f'\tlower quartile: [{pheno_cohort[pheno_field].values.min()},{lower}]')
                print(f'\tupper quartile: ({upper},{pheno_cohort[pheno_field].values.max()}]')
            samples_lower.extend(pheno_cohort.loc[pheno_cohort[pheno_field] <= lower].index.tolist())
            samples_upper.extend(pheno_cohort.loc[pheno_cohort[pheno_field] > upper].index.tolist())
        if len(samples_lower) >= thresh and len(samples_upper) >= thresh:
            printer.append((samples_lower, 'lower'))
            printer.append((samples_upper, 'upper'))
    return printer

def stage_conf_chi(pheno):
    confs = ['stage', 'age_quartile', 'age', 'ethnicity', 'sex']
    sc = {'age_quartile':np.nan, 'age':np.nan, 'ethnicity':np.nan, 'sex':np.nan}
    for conf in confs:
        out = []
        confusion_table = pd.DataFrame()
        try:
            orig_conf_partition = get_conf_partition(pheno, conf, 0, thresh=0, verbose=True)
        except KeyError:
                continue
        for bl in orig_conf_partition:
            block = bl[0]
            print(bl[1])
            print(len(block))
            try:
                stage_partition = get_conf_partition(pheno[pheno.index.isin(block)], 'stage', 0)
            except:
                continue
            for stage_block in stage_partition:
                confusion_table.loc[stage_block[1], bl[1]] = len(stage_block[0])
        try:
            if len(confusion_table) <= 1:
                continue
            confusion_table = confusion_table.dropna()
            chi2, p, dof, ex = chi2_contingency(confusion_table, correction=False)
            sc[str(conf)] = p
        except:
            continue
    return sc

def get_expression_data(cohort):
    try:
        expression_data = pd.read_csv(os.path.join(os.getcwd(), 'datasets', f'TCGA-{cohort}.htseq_fpkm.tsv'), sep='\t', header=0, index_col=0)
    except:
        expression_data = pd.read_csv(os.path.join(os.getcwd(), 'datasets', 'brca_metabric.tsv'), sep='\t', header=0, index_col=0)
    expression_data.columns = expression_data.columns.str.split('.').str[0].tolist()
    # gene filtering is removed here, because it does not affect the set of samples present in the file
    return expression_data

def get_pheno_data(cohort):
    try:
        tissue_type_field, tissue_type = 'sample_type.samples', 'Primary Tumor'
        pheno_data = pd.read_csv(os.path.join(os.getcwd(), 'datasets', f'TCGA-{cohort}.GDC_phenotype.tsv'), sep='\t', header=0, index_col='submitter_id.samples')
    except:
        tissue_type_field, tissue_type = 'Sample Type', 'Primary'
        pheno_data = pd.read_csv(os.path.join(os.getcwd(), 'datasets', 'brca_metabric_pheno.tsv'), sep='\t', header=0, index_col=0)
    assert len(pheno_data.iloc[0]) == len(pheno_data.iloc[0].values)
    pheno_data['cohort'] = str(cohort)
    pheno_data =  pheno_data[pheno_data[tissue_type_field] == tissue_type]
    return pheno_data


df = pd.DataFrame(columns=['cohort', 'age_quartile', 'age', 'ethnicity', 'sex'])
for cohort in cohorts:
    print(cohort)
    # read single data
    expr = get_expression_data(cohort)
    pheno = get_pheno_data(cohort)
    # align pheno and expr
    pheno, expr = align(pheno, expr)
    sc = stage_conf_chi(pheno)
    temp = pd.DataFrame({'cohort':[str(cohort)], 'age_quartile':[sc['age_quartile']], 'age':[sc['age']], 'ethnicity':[sc['ethnicity']], 'sex':[sc['sex']]})
    df = pd.concat([df, temp])
    print('\n')
    
df_work = df.copy()
#df_work = df_work.dropna().reset_index()
df_work = df_work.reset_index()
df_work.index = df_work['cohort'].values
df_work = df_work.drop(['cohort', 'index'], axis=1)
df_work.columns = ['age_quartile', 'age', 'ethnicity', 'sex']

brca_metabric
na:
300
0.0
5
na:
0
1.0
225
na:
0
2.0
484
na:
0
3.0
73
na:
0
4.0
4
na:
0
na:
0
lower
379
na:
85
upper
712
na:
215
na:
na:
na:




In [4]:
df_work
df_work.to_csv(os.path.join(cwd, 'chi2_pvals_stage.csv'))

In [5]:
def stage_conf_chi(pheno):
    confs = ['age_quartile', 'age', 'ethnicity', 'sex']
    sc = {'age_quartile':np.nan, 'age':np.nan, 'ethnicity':np.nan, 'sex':np.nan}
    for conf in confs:
        out = []
        confusion_table = pd.DataFrame()
        try:
            orig_conf_partition = get_conf_partition(pheno, conf, 0, thresh=20, verbose=True)
        except KeyError:
                print(f'no {conf}!')
                continue
        for bl in orig_conf_partition:
            block = bl[0]
            try:
                stage_partition = get_conf_partition(pheno[pheno.index.isin(block)], 'cigarettes_per_day.exposures', 0)
            except:
                print('no stage variable')
                continue
            for stage_block in stage_partition:
                confusion_table.loc[stage_block[1], bl[1]] = len(stage_block[0])
        try:
            if len(confusion_table) <= 1:
                continue
            confusion_table = confusion_table.dropna()
            chi2, p, dof, ex = chi2_contingency(confusion_table, correction=False)
            sc[str(conf)] = p
        except:
            continue
    print(sc)
    return sc

df = pd.DataFrame(columns=['cohort', 'age_quartile', 'age', 'ethnicity', 'sex'])
for cohort in cohorts:
    print(cohort)
    # read single data
    expr = get_expression_data(cohort)
    pheno = get_pheno_data(cohort)
    # align pheno and expr
    pheno, expr = align(pheno, expr)
    sc = stage_conf_chi(pheno)
    temp = pd.DataFrame({'cohort':[str(cohort)], 'age_quartile':[sc['age_quartile']], 'age':[sc['age']], 'ethnicity':[sc['ethnicity']], 'sex':[sc['sex']]})
    df = pd.concat([df, temp])
    print('\n')
    
df_work = df.copy()
#df_work = df_work.dropna().reset_index()
df_work = df_work.reset_index()
df_work.index = df_work['cohort'].values
df_work = df_work.drop(['cohort', 'index'], axis=1)
df_work.columns = ['age_quartile', 'age', 'ethnicity', 'sex']

LUAD
na:
no age_quartile!
na:
19
	lower quartile: [33.0,59.0]
	upper quartile: (72.0,88.0]
na:
42
na:
42
na:
66
na:
20
na:
124
na:
0
na:
93
na:
70
{'age_quartile': nan, 'age': 0.926028073733714, 'ethnicity': 0.7262204204251024, 'sex': 0.41707894453112837}


PRAD
na:
no age_quartile!
na:
0
	lower quartile: [41,56.0]
	upper quartile: (66.0,78]
na:
no stage variable
na:
no stage variable
na:
14
na:
no stage variable
na:
no stage variable
na:
0
na:
no stage variable
{'age_quartile': nan, 'age': nan, 'ethnicity': nan, 'sex': nan}




In [6]:
df_work.to_csv(os.path.join(cwd, 'chi2_pvals_cigarettes_per_day.exposures.csv'))

In [7]:
def stage_conf_chi(pheno):
    confs = ['age_quartile', 'age', 'ethnicity', 'sex']
    sc = {'age_quartile':np.nan, 'age':np.nan, 'ethnicity':np.nan, 'sex':np.nan}
    for conf in confs:
        out = []
        confusion_table = pd.DataFrame()
        try:
            orig_conf_partition = get_conf_partition(pheno, conf, 0, thresh=20, verbose=True)
        except KeyError:
                print(f'no {conf}!')
                continue
        for bl in orig_conf_partition:
            block = bl[0]
            try:
                stage_partition = get_conf_partition(pheno[pheno.index.isin(block)], 'alcohol_history.exposures', 0)
            except:
                print('no stage variable')
                continue
            for stage_block in stage_partition:
                confusion_table.loc[stage_block[1], bl[1]] = len(stage_block[0])
        try:
            if len(confusion_table) <= 1:
                continue
            confusion_table = confusion_table.dropna()
            chi2, p, dof, ex = chi2_contingency(confusion_table, correction=False)
            sc[str(conf)] = p
        except:
            continue
    print(sc)
    return sc

df = pd.DataFrame(columns=['cohort', 'age_quartile', 'age', 'ethnicity', 'sex'])
for cohort in cohorts:
    print(cohort)
    # read single data
    expr = get_expression_data(cohort)
    pheno = get_pheno_data(cohort)
    # align pheno and expr
    pheno, expr = align(pheno, expr)
    sc = stage_conf_chi(pheno)
    temp = pd.DataFrame({'cohort':[str(cohort)], 'age_quartile':[sc['age_quartile']], 'age':[sc['age']], 'ethnicity':[sc['ethnicity']], 'sex':[sc['sex']]})
    df = pd.concat([df, temp])
    print('\n')
    
df_work = df.copy()
#df_work = df_work.dropna().reset_index()
df_work = df_work.reset_index()
df_work.index = df_work['cohort'].values
df_work = df_work.drop(['cohort', 'index'], axis=1)
df_work.columns = ['age_quartile', 'age', 'ethnicity', 'sex']

LUAD
na:
no age_quartile!
na:
19
	lower quartile: [33.0,59.0]
	upper quartile: (72.0,88.0]
na:
0
na:
0
na:
66
na:
0
na:
0
na:
0
na:
0
na:
0
{'age_quartile': nan, 'age': nan, 'ethnicity': nan, 'sex': nan}


PRAD
na:
no age_quartile!
na:
0
	lower quartile: [41,56.0]
	upper quartile: (66.0,78]
na:
0
na:
0
na:
14
na:
0
na:
0
na:
0
na:
0
{'age_quartile': nan, 'age': nan, 'ethnicity': nan, 'sex': nan}




In [8]:
df_work.to_csv(os.path.join(cwd, 'chi2_pvals_alcohol_history.exposures.csv'))